In [1]:
# load tydi json and compute stats on the questions and answers
import glob
import gzip
import json

train_data_dir = "/dccstor/srosent2/primeqa/data/train/nq-lfqa/nq-train*jsonl"
dev_data_dir = "/dccstor/srosent2/primeqa/data/dev/nq-lfqa/*jsonl"

train_files = glob.glob(train_data_dir)
dev_files = glob.glob(dev_data_dir)

In [2]:

def load_json_from_file(gt_file_patterns):
    data = []
    if gt_file_patterns.endswith('gz'):
        f = gzip.open(gt_file_patterns, 'rt', encoding='utf-8')
    else:
        f = open(gt_file_patterns, 'rt', encoding='utf-8')
    lines = f.readlines()
    for line in lines:
        data.append(json.loads(line))
    return data


In [37]:
from random import sample

def compute_stats(data, type):
    first_word_stat = {}
    for example in data:
        qtype = example['type'][0]
        
        if qtype != type:
            continue
        example_id = example['example_id']
        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        info = {}
        info["question"] = question
        info["passage_text"] = passage_text
        info["minimal_text"] = minimal_text
        info["example_id"] = example_id
        if q_word in first_word_stat:
            first_word_stat[q_word].append(info)
        else:
            first_word_stat[q_word] = []
            first_word_stat[q_word].append(info)
    for word in first_word_stat:
        if len(first_word_stat[word]) > 20:
            sample_questions = sample(first_word_stat[word],5)
            print(word + ": " + str(len(first_word_stat[word])))
            for sample_question in sample_questions:
                print(str(sample_question['example_id']) + "\t" + sample_question['question'] + "\t" + sample_question['passage_text'] + "\t" + sample_question['minimal_text'])
                    

In [10]:
dev_data = []
for dev_file in dev_files:
    dev_data.extend(load_json_from_file(dev_file))

compute_stats(dev_data, "la")

what: 159
how: 54
when: 57
who: 140
is: 30
where: 100
the: 24


In [35]:
train_data = []
for train_file in train_files:
    train_data.extend(load_json_from_file(train_file))

In [38]:
compute_stats(train_data, "la")

what: 9832
70293327710385289	what age is statutory rape in south africa	b"The age of consent in South Africa is 16 , as specified by sections 15 and 16 of the Criminal Law ( Sexual Offences and Related Matters ) Amendment Act , 2007 . Section 15 ( `` statutory rape '' ) prohibits the commission of `` an act of sexual penetration with a child who is 12 years of age or older but under the age of 16 years '' , while section 16 ( `` statutory sexual assault '' ) prohibits the commission of `` an act of sexual violation with a child who is 12 years of age or older but under the age of 16 years '' . The law includes a close - in - age exception , so that sexual acts between two children where both are between 12 and 16 , or where one is under 16 and the other is less than two years older , are not criminal . Children under the age of 12 are conclusively presumed by the law to be incapable of consenting , so a sexual act with a child under that age constitutes rape or sexual assault ."	b"12 y